In [14]:
# pip install midi
from midiutil.MidiFile import MIDIFile
import os

START_OF_ARRAY = 41     # Bottom G note. 
TOP_OF_ARRAY = 79       # Top G note. 

note_dict = {
    "G": 43,
    "A": 45,
    "B": 47,
    "C": 48,
    "D": 50,
    "E": 52,
    "F": 53,
}

octave_dict = {
    2: 0,
    3: 12,
    4: 24,
    5: 36,
    6: 48, 
    7: 60,
}

def string_parse(in_str):
    duration = in_str[1]
    note = in_str[0]
    sharp = note[1]
    octave = note[-1]

    midi_value = note_dict[note[0]]
    
    if sharp =="#":
        midi_value +=1
    midi_value += octave_dict[int(octave)]

    return duration, midi_value

def populate_midi(notes, tempo):
    path = "output.midi"
    mf = MIDIFile(1)     # only 1 track
    track = 0   # the only track
    time = 0    # start at the beginning
    channel = 0
    volume = 100
    tempo = int(tempo)
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, tempo)
    for note_time, pitches in enumerate(notes):
        for pitch in pitches:
            duration, note_value = string_parse(pitch)
            print(note_time)
            mf.addNote(track, channel, note_value, note_time, duration, volume)
    
    with open(path, 'wb') as f: 
        mf.writeFile(f) 
    
    #descriptor = os.open(path=path, 
    #                     flags=(
    #                         os.O_WRONLY 
    #                         | os.O_CREAT
    #                         | os.O_TRUNC
    #                     ),
    #                     mode=0o777)

    #with open(descriptor, 'wb') as outf:
    #    mf.writeFile(outf)
    print(mf)
    return "output.midi"


In [20]:
notes = string
tempo = 200
path = "output.midi"
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
channel = 0
volume = 100
tempo = int(tempo)
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, tempo)
for note_time, pitches in enumerate(notes):
    for pitch in pitches:
        duration, note_value = string_parse(pitch)
        mf.addNote(track, channel, note_value, note_time, duration, volume)
    


In [15]:
string =[[['G3', '1'], ['G2', '8']], [['B3', '1']], [['G4', '1']], [['B3', '1']], [['G5', '2'], ['G3', '1']], [['B3', '1']], [['G5', '2'], ['G4', '1']], [['B3', '1']], [['F#5', '3'], ['F#3', '1'], ['D3', '8']], [['A3', '1']], [['F#4', '1']], [['E5', '1'], ['A3', '1']], [['F#5', '2'], ['F#3', '1']], [['A3', '1']], [['F#5', '1'], ['F#4', '1']], [['G5', '1'], ['A3', '1']], [['E5', '11'], ['E3', '8']], [['G3', '1']], [['E4', '1']], [['G3', '1']], [], [['G3', '1']], [['E4', '1']], [['G3', '1']], [['E3', '1'], ['C3', '8']], [['C4', '1']], [['E4', '1']], [['C4', '1']], [['E5', '2'], ['E3', '1']], [['C4', '1']], [['F#5', '2'], ['E4', '1']], [['C4', '1']]]

In [16]:
print(string)

[[['G3', '1'], ['G2', '8']], [['B3', '1']], [['G4', '1']], [['B3', '1']], [['G5', '2'], ['G3', '1']], [['B3', '1']], [['G5', '2'], ['G4', '1']], [['B3', '1']], [['F#5', '3'], ['F#3', '1'], ['D3', '8']], [['A3', '1']], [['F#4', '1']], [['E5', '1'], ['A3', '1']], [['F#5', '2'], ['F#3', '1']], [['A3', '1']], [['F#5', '1'], ['F#4', '1']], [['G5', '1'], ['A3', '1']], [['E5', '11'], ['E3', '8']], [['G3', '1']], [['E4', '1']], [['G3', '1']], [], [['G3', '1']], [['E4', '1']], [['G3', '1']], [['E3', '1'], ['C3', '8']], [['C4', '1']], [['E4', '1']], [['C4', '1']], [['E5', '2'], ['E3', '1']], [['C4', '1']], [['F#5', '2'], ['E4', '1']], [['C4', '1']]]


In [17]:
mf

NameError: name 'mf' is not defined

In [18]:
populate_midi(string, 180)

0
0
1
2
3
4
4
5
6
6
7
8
8
8
9
10
11
11
12
12
13
14
14
15
15
16
16
17
18
19
21
22
23
24
24
25
26
27
28
28
29
30
30
31


'output.midi'

In [12]:
from midiutil import MIDIFile

degrees  = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
track    = 0
channel  = 0
time     = 0   # In beats
duration = 1   # In beats
tempo    = 60  # In BPM
volume   = 100 # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                     # automatically created)
MyMIDI.addTempo(track,time, tempo)

for pitch in degrees:
    MyMIDI.addNote(track, channel, pitch, time, duration, volume)
    time = time + 1
    print(time)

with open("major-scale.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

1
2
3
4
5
6
7
8
